In [1]:
import matplotlib.pyplot as plt
plt.switch_backend('tkagg')
from pycigar.utils.input_parser import input_parser
import numpy as np
from pycigar.utils.registry import register_devcon
import tensorflow as tf
from ray.rllib.models.catalog import ModelCatalog
from gym.spaces import Tuple, Discrete, Box
import matplotlib
%matplotlib inline

# misc_inputs = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/misc_inputs.csv'
# dss = '/home/toanngo/Documents/GitHub/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/ieee37.dss'
# load_solar = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/load_solar_data.csv'
# breakpoints = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/breakpoints.csv'

misc_inputs = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee123busdata/misc_inputs.csv'
dss = '/home/toanngo/Documents/GitHub/ceds-cigar/pycigar/data/ieee123busdata/ieee123.dss'
load_solar = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee123busdata/load_solar_data.csv'
breakpoints = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee123busdata/breakpoints.csv'

start = 100
hack = 0
sim_params = input_parser(misc_inputs, dss, load_solar, breakpoints, benchmark=True, vectorized_mode=True, percentage_hack=hack)
sim_params['scenario_config']['start_end_time'] = [start, start + 750]
sim_params['env_config']['sims_per_step'] = 1
sim_params['scenario_config']['custom_configs']['solar_scaling_factor'] = 0
del sim_params['attack_randomization']
for node in sim_params['scenario_config']['nodes']:
    node['devices'][0]['adversary_controller'] =  'adaptive_unbalanced_fixed_controller'

from pycigar.envs import CentralControlPhaseSpecificPVInverterEnv
env = CentralControlPhaseSpecificPVInverterEnv(sim_params=sim_params)

qv_matrix = {}
#base voltage 
env.reset()
env.step([10, 10, 10])
base_voltage = env.unwrapped.k.kernel_api.puvoltage
for device_id in env.unwrapped.k.device.devices.keys():
    if 'adversary' not in device_id and 'reg' not in device_id:
        env.reset()
        node_id = env.unwrapped.k.device.devices[device_id]['node_id']
        env.unwrapped.k.node.nodes[node_id]['PQ_injection']['Q'] = 1
        env.step([10, 10, 10])
        new_voltage = env.unwrapped.k.kernel_api.puvoltage
        qv_matrix[device_id] =  np.array(new_voltage) - np.array(base_voltage)

Instructions for updating:
non-resource variables are not supported in the long term
/home/toanngo/Documents/GitHub/ceds-cigar/pycigar/devices/vectorized_pv_inverter_device.py:210: RuntimeWarning: invalid value encountered in true_divide
  Logger.log(device_id, 'sbar_pset', self.p_set[i] / self.Sbar[i])
/home/toanngo/Documents/GitHub/ceds-cigar/pycigar/devices/vectorized_pv_inverter_device.py:210: RuntimeWarning: invalid value encountered in double_scalars
  Logger.log(device_id, 'sbar_pset', self.p_set[i] / self.Sbar[i])


In [2]:
env.unwrapped.k.kernel_api.all_bus_name

['150',
 '149',
 '1',
 '2',
 '3',
 '7',
 '4',
 '5',
 '6',
 '8',
 '12',
 '9',
 '13',
 '9r',
 '14',
 '34',
 '18',
 '11',
 '10',
 '15',
 '16',
 '17',
 '19',
 '21',
 '20',
 '22',
 '23',
 '24',
 '25',
 '25r',
 '26',
 '28',
 '27',
 '31',
 '33',
 '29',
 '30',
 '250',
 '32',
 '35',
 '36',
 '40',
 '37',
 '38',
 '39',
 '41',
 '42',
 '43',
 '44',
 '45',
 '47',
 '46',
 '48',
 '49',
 '50',
 '51',
 '151',
 '52',
 '53',
 '54',
 '55',
 '57',
 '56',
 '58',
 '60',
 '59',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '72',
 '97',
 '69',
 '70',
 '71',
 '73',
 '76',
 '74',
 '75',
 '77',
 '86',
 '78',
 '79',
 '80',
 '81',
 '82',
 '84',
 '83',
 '85',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '98',
 '99',
 '100',
 '450',
 '197',
 '101',
 '102',
 '105',
 '103',
 '104',
 '106',
 '108',
 '107',
 '109',
 '300',
 '110',
 '111',
 '112',
 '113',
 '114',
 '135',
 '152',
 '160r',
 '150r',
 '160',
 '61s',
 '300_open',
 '94_open',
 '610']

In [3]:
import pandas as pd

df = pd.DataFrame.from_dict(qv_matrix)
df = df.set_index([list(env.unwrapped.k.kernel_api.offsets.keys())])

In [4]:
df

,inverter_s1a,inverter_s2b,inverter_s4c,inverter_s5c,inverter_s6c,inverter_s7a,inverter_s9a,inverter_s10a,inverter_s11a,inverter_s12b,...,inverter_s102c,inverter_s103c,inverter_s104c,inverter_s106b,inverter_s107b,inverter_s109a,inverter_s111a,inverter_s112a,inverter_s113a,inverter_s114a
150.1,-1.729166e-08,2.085203e-10,-1.976790e-10,-1.977385e-10,-1.977916e-10,-1.751138e-08,-1.766024e-08,-1.766540e-08,-1.766056e-08,4.632290e-10,...,-1.175325e-09,-1.180300e-09,-1.185306e-09,1.160781e-09,1.158903e-09,-1.858338e-08,-1.861345e-08,-1.859936e-08,-1.860422e-08,-1.861454e-08
150.2,-1.741285e-10,-1.687646e-08,6.949774e-11,6.935896e-11,6.923928e-11,-3.068481e-10,-3.980187e-10,-3.981513e-10,-3.980134e-10,-1.671734e-08,...,4.264240e-10,4.282406e-10,4.300573e-10,-1.622870e-08,-1.622142e-08,-1.083340e-09,-1.085461e-09,-1.084568e-09,-1.085049e-09,-1.085738e-09
150.3,1.313947e-10,-1.722871e-10,-1.717359e-08,-1.717284e-08,-1.717227e-08,2.293274e-10,2.953906e-10,2.950939e-10,2.948639e-10,-3.686078e-10,...,-1.787196e-08,-1.790757e-08,-1.794342e-08,-1.036529e-09,-1.037495e-09,9.009904e-10,9.020138e-10,9.010883e-10,9.006254e-10,9.011423e-10
149.1,-2.458279e-08,2.955935e-10,-2.802476e-10,-2.803324e-10,-2.804081e-10,-2.489306e-08,-2.510316e-08,-2.511015e-08,-2.510310e-08,6.568068e-10,...,-1.667573e-09,-1.674689e-09,-1.681850e-09,1.648661e-09,1.646000e-09,-2.640383e-08,-2.644651e-08,-2.642629e-08,-2.643289e-08,-2.644761e-08
149.2,-2.472522e-10,-2.399123e-08,9.765055e-11,9.745338e-11,9.728307e-11,-4.356786e-10,-5.651020e-10,-5.652872e-10,-5.650878e-10,-2.375966e-08,...,5.977960e-10,6.003598e-10,6.029230e-10,-2.304117e-08,-2.303036e-08,-1.538370e-09,-1.541392e-09,-1.540113e-09,-1.540789e-09,-1.541772e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300_open.3,4.134306e-06,1.987006e-07,-1.285620e-05,-1.284891e-05,-1.284269e-05,7.431606e-06,9.732501e-06,9.744404e-06,9.746204e-06,3.026345e-07,...,-3.873889e-05,-3.883073e-05,-3.892297e-05,-2.781922e-06,-2.816501e-06,1.535775e-05,1.538374e-05,1.537889e-05,1.539326e-05,1.539935e-05
94_open.1,-1.351284e-05,4.972477e-06,4.297918e-07,4.242305e-07,4.194927e-07,-2.391851e-05,-3.099298e-05,-3.098098e-05,-3.096359e-05,1.139917e-05,...,-7.570321e-07,-7.415623e-07,-7.261726e-07,2.762542e-05,2.765887e-05,-6.920522e-05,-6.931616e-05,-6.924370e-05,-6.923310e-05,-6.927899e-05
610.1,-5.553214e-06,-1.553117e-06,-8.319717e-07,-8.288052e-07,-8.261094e-07,-9.810839e-06,-1.269932e-05,-1.269234e-05,-1.268484e-05,-3.549615e-06,...,-5.006545e-06,-5.057478e-06,-5.108555e-06,-1.510152e-05,-1.512444e-05,-4.516284e-05,-4.523847e-05,-4.519029e-05,-4.518478e-05,-4.521562e-05
610.2,-2.707441e-07,-4.567031e-06,-2.080615e-06,-2.082249e-06,-2.083646e-06,-4.240043e-07,-4.984710e-07,-4.902372e-07,-4.864001e-07,-1.024671e-05,...,-1.716468e-05,-1.718591e-05,-1.720741e-05,-3.536890e-05,-3.532001e-05,-1.460811e-06,-1.462504e-06,-1.442884e-06,-1.416941e-06,-1.424727e-06


In [22]:
df = df[df.index.isin(full_nodes)]
data = df.to_numpy()



In [26]:
from sklearn.cluster import SpectralClustering
clustering = SpectralClustering(n_clusters=2, assign_labels='kmeans', random_state=42, gamma=1).fit(data)

In [27]:
clustering.labels_.shape

(95,)

In [28]:
print(df.index[clustering.labels_ == 0])
print(df.index[clustering.labels_ == 1])
print(df.index[clustering.labels_ == 2])
print(df.index[clustering.labels_ == 3])
print(df.index[clustering.labels_ == 4])

Index(['1.1', '2.2', '4.3', '5.3', '6.3', '12.2', '34.3', '16.3', '17.3',
       '22.2', '24.3', '31.3', '30.3', '32.3', '38.2', '39.2', '41.3', '43.2',
       '47.2', '47.3', '48.2', '48.3', '49.2', '49.3', '50.3', '56.2', '58.2',
       '59.2', '62.3', '64.2', '65.2', '65.3', '66.3', '73.3', '76.2', '76.3',
       '74.3', '75.3', '77.2', '86.2', '80.2', '84.3', '83.3', '85.3', '87.2',
       '90.2', '92.3', '95.2', '96.2', '99.2', '100.3', '102.3', '103.3',
       '104.3', '106.2', '107.2'],
      dtype='object')
Index(['7.1', '9.1', '11.1', '10.1', '19.1', '20.1', '28.1', '33.1', '29.1',
       '35.1', '37.1', '42.1', '45.1', '47.1', '46.1', '48.1', '49.1', '51.1',
       '52.1', '53.1', '55.1', '60.1', '63.1', '65.1', '68.1', '69.1', '70.1',
       '71.1', '76.1', '79.1', '82.1', '88.1', '94.1', '98.1', '109.1',
       '111.1', '112.1', '113.1', '114.1'],
      dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')


In [9]:
df.columns

Index(['inverter_s1a', 'inverter_s2b', 'inverter_s4c', 'inverter_s5c',
       'inverter_s6c', 'inverter_s7a', 'inverter_s9a', 'inverter_s10a',
       'inverter_s11a', 'inverter_s12b', 'inverter_s16c', 'inverter_s17c',
       'inverter_s19a', 'inverter_s20a', 'inverter_s22b', 'inverter_s24c',
       'inverter_s28a', 'inverter_s29a', 'inverter_s30c', 'inverter_s31c',
       'inverter_s32c', 'inverter_s33a', 'inverter_s34c', 'inverter_s35a',
       'inverter_s37a', 'inverter_s38b', 'inverter_s39b', 'inverter_s41c',
       'inverter_s42a', 'inverter_s43b', 'inverter_s45a', 'inverter_s46a',
       'inverter_s47', 'inverter_s48', 'inverter_s49a', 'inverter_s49b',
       'inverter_s49c', 'inverter_s50c', 'inverter_s51a', 'inverter_s52a',
       'inverter_s53a', 'inverter_s55a', 'inverter_s56b', 'inverter_s58b',
       'inverter_s59b', 'inverter_s60a', 'inverter_s62c', 'inverter_s63a',
       'inverter_s64b', 'inverter_s65a', 'inverter_s65b', 'inverter_s65c',
       'inverter_s66c', 'inverter_

In [12]:
print(df.index[clustering.labels_ == 0])
print(df.index[clustering.labels_ == 1])
print(df.index[clustering.labels_ == 2])
print(df.index[clustering.labels_ == 3])
print(df.index[clustering.labels_ == 4])
clusters = {}
for inv in df.columns:
    inv = inv.split('_')[1][1:]
    if inv[-1].isdigit():
        bus = inv
        phase = ''
    else:
        phase = inv[-1]
        bus = inv[:-1]
    if phase == '':
        for p in ['1', '2', '3']:
            node = '.'.join([bus, p])
            for cluster in [0, 1, 2, 3, 4]:
                if node in df.index[clustering.labels_ == cluster]:
                    if cluster not in clusters:
                        clusters[cluster] = [node]
                    clusters[cluster].append(node)
    else:
        if phase == 'a':
            phase = '1'
        if phase == 'b':
            phase = '2'
        if phase == 'c':
            phase = '3'
        node = '.'.join([bus, phase])
        for cluster in [0, 1, 2, 3, 4]:
            if node in df.index[clustering.labels_ == cluster]:
                if cluster not in clusters:
                    clusters[cluster] = [node]
                clusters[cluster].append(node)

Index(['7.1', '8.1', '9.1', '13.1', '9r.1', '14.1', '18.1', '11.1', '10.1',
       '19.1', '21.1', '20.1', '23.1', '25.1', '25r.1', '26.1', '28.1', '27.1',
       '33.1', '29.1', '30.1', '250.1', '35.1', '36.1', '40.1', '37.1', '42.1',
       '44.1', '45.1', '47.1', '46.1', '48.1', '49.1', '50.1', '51.1', '151.1',
       '52.1', '53.1', '54.1', '55.1', '56.1', '135.1', '152.1', '300_open.1',
       '94_open.1'],
      dtype='object')
Index(['7.2', '8.2', '12.2', '13.2', '18.2', '21.2', '22.2', '23.2', '25.2',
       '28.2', '29.2', '30.2', '250.2', '35.2', '36.2', '40.2', '38.2', '39.2',
       '42.2', '43.2', '44.2', '47.2', '48.2', '49.2', '50.2', '51.2', '151.2',
       '52.2', '53.2', '54.2', '55.2', '57.2', '56.2', '58.2', '60.2', '59.2',
       '61.2', '62.2', '63.2', '64.2', '65.2', '66.2', '67.2', '72.2', '97.2',
       '76.2', '77.2', '86.2', '78.2', '79.2', '80.2', '81.2', '82.2', '83.2',
       '87.2', '89.2', '90.2', '91.2', '93.2', '95.2', '96.2', '98.2', '99.2',
       '1

In [16]:
full_nodes = []
for val in clusters.values():
    full_nodes.extend(val)

In [20]:
full_nodes = list(set(full_nodes))

In [19]:
df

,inverter_s1a,inverter_s2b,inverter_s4c,inverter_s5c,inverter_s6c,inverter_s7a,inverter_s9a,inverter_s10a,inverter_s11a,inverter_s12b,...,inverter_s102c,inverter_s103c,inverter_s104c,inverter_s106b,inverter_s107b,inverter_s109a,inverter_s111a,inverter_s112a,inverter_s113a,inverter_s114a
150.1,-1.729166e-08,2.085203e-10,-1.976790e-10,-1.977385e-10,-1.977916e-10,-1.751138e-08,-1.766024e-08,-1.766540e-08,-1.766056e-08,4.632290e-10,...,-1.175325e-09,-1.180300e-09,-1.185306e-09,1.160781e-09,1.158903e-09,-1.858338e-08,-1.861345e-08,-1.859936e-08,-1.860422e-08,-1.861454e-08
150.2,-1.741285e-10,-1.687646e-08,6.949774e-11,6.935896e-11,6.923928e-11,-3.068481e-10,-3.980187e-10,-3.981513e-10,-3.980134e-10,-1.671734e-08,...,4.264240e-10,4.282406e-10,4.300573e-10,-1.622870e-08,-1.622142e-08,-1.083340e-09,-1.085461e-09,-1.084568e-09,-1.085049e-09,-1.085738e-09
150.3,1.313947e-10,-1.722871e-10,-1.717359e-08,-1.717284e-08,-1.717227e-08,2.293274e-10,2.953906e-10,2.950939e-10,2.948639e-10,-3.686078e-10,...,-1.787196e-08,-1.790757e-08,-1.794342e-08,-1.036529e-09,-1.037495e-09,9.009904e-10,9.020138e-10,9.010883e-10,9.006254e-10,9.011423e-10
149.1,-2.458279e-08,2.955935e-10,-2.802476e-10,-2.803324e-10,-2.804081e-10,-2.489306e-08,-2.510316e-08,-2.511015e-08,-2.510310e-08,6.568068e-10,...,-1.667573e-09,-1.674689e-09,-1.681850e-09,1.648661e-09,1.646000e-09,-2.640383e-08,-2.644651e-08,-2.642629e-08,-2.643289e-08,-2.644761e-08
149.2,-2.472522e-10,-2.399123e-08,9.765055e-11,9.745338e-11,9.728307e-11,-4.356786e-10,-5.651020e-10,-5.652872e-10,-5.650878e-10,-2.375966e-08,...,5.977960e-10,6.003598e-10,6.029230e-10,-2.304117e-08,-2.303036e-08,-1.538370e-09,-1.541392e-09,-1.540113e-09,-1.540789e-09,-1.541772e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300_open.3,4.134306e-06,1.987006e-07,-1.285620e-05,-1.284891e-05,-1.284269e-05,7.431606e-06,9.732501e-06,9.744404e-06,9.746204e-06,3.026345e-07,...,-3.873889e-05,-3.883073e-05,-3.892297e-05,-2.781922e-06,-2.816501e-06,1.535775e-05,1.538374e-05,1.537889e-05,1.539326e-05,1.539935e-05
94_open.1,-1.351284e-05,4.972477e-06,4.297918e-07,4.242305e-07,4.194927e-07,-2.391851e-05,-3.099298e-05,-3.098098e-05,-3.096359e-05,1.139917e-05,...,-7.570321e-07,-7.415623e-07,-7.261726e-07,2.762542e-05,2.765887e-05,-6.920522e-05,-6.931616e-05,-6.924370e-05,-6.923310e-05,-6.927899e-05
610.1,-5.553214e-06,-1.553117e-06,-8.319717e-07,-8.288052e-07,-8.261094e-07,-9.810839e-06,-1.269932e-05,-1.269234e-05,-1.268484e-05,-3.549615e-06,...,-5.006545e-06,-5.057478e-06,-5.108555e-06,-1.510152e-05,-1.512444e-05,-4.516284e-05,-4.523847e-05,-4.519029e-05,-4.518478e-05,-4.521562e-05
610.2,-2.707441e-07,-4.567031e-06,-2.080615e-06,-2.082249e-06,-2.083646e-06,-4.240043e-07,-4.984710e-07,-4.902372e-07,-4.864001e-07,-1.024671e-05,...,-1.716468e-05,-1.718591e-05,-1.720741e-05,-3.536890e-05,-3.532001e-05,-1.460811e-06,-1.462504e-06,-1.442884e-06,-1.416941e-06,-1.424727e-06


In [21]:
df[df.index.isin(full_nodes)]

,inverter_s1a,inverter_s2b,inverter_s4c,inverter_s5c,inverter_s6c,inverter_s7a,inverter_s9a,inverter_s10a,inverter_s11a,inverter_s12b,...,inverter_s102c,inverter_s103c,inverter_s104c,inverter_s106b,inverter_s107b,inverter_s109a,inverter_s111a,inverter_s112a,inverter_s113a,inverter_s114a
1.1,-0.000013,4.802333e-06,6.020252e-07,5.964625e-07,5.917577e-07,-0.000013,-1.351468e-05,-1.351031e-05,-1.350233e-05,5.031024e-06,...,-5.220650e-07,-5.202245e-07,-5.184105e-07,5.902873e-06,5.908880e-06,-1.377858e-05,-1.379955e-05,-1.378434e-05,-1.378022e-05,-1.378948e-05
2.2,0.000001,-1.983045e-05,4.310531e-06,4.314183e-06,4.317286e-06,0.000001,9.889675e-07,9.797381e-07,9.746400e-07,-1.227943e-05,...,4.733658e-06,4.740785e-06,4.747974e-06,-1.118204e-05,-1.116513e-05,1.012091e-07,9.948251e-08,9.435252e-08,8.580664e-08,8.748832e-08
7.1,-0.000013,4.824927e-06,5.652642e-07,5.597212e-07,5.550245e-07,-0.000024,-2.380584e-05,-2.379878e-05,-2.378506e-05,8.707505e-06,...,-7.804477e-07,-7.769100e-07,-7.734168e-07,1.026170e-05,1.027251e-05,-2.430278e-05,-2.433988e-05,-2.431341e-05,-2.430674e-05,-2.432294e-05
4.3,0.000004,6.928261e-07,-3.182547e-05,-2.342644e-05,-2.341138e-05,0.000004,4.384281e-06,4.390190e-06,4.391369e-06,3.540906e-07,...,-1.289840e-05,-1.293122e-05,-1.296420e-05,-8.705053e-07,-8.822578e-07,5.248534e-06,5.257370e-06,5.255923e-06,5.261091e-06,5.263086e-06
5.3,0.000004,6.917470e-07,-2.340777e-05,-3.700181e-05,-3.697439e-05,0.000004,4.377439e-06,4.383339e-06,4.384516e-06,3.535440e-07,...,-1.287835e-05,-1.291111e-05,-1.294404e-05,-8.691341e-07,-8.808683e-07,5.240360e-06,5.249182e-06,5.247738e-06,5.252898e-06,5.254889e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109.1,-0.000015,5.681103e-06,3.508619e-07,3.446283e-07,3.392910e-07,-0.000027,-3.465092e-05,-3.463332e-05,-3.461434e-05,1.301349e-05,...,3.912333e-06,3.969808e-06,4.027293e-06,5.586792e-05,5.594131e-05,-1.910784e-04,-1.914375e-04,-1.912331e-04,-1.912408e-04,-1.913779e-04
111.1,-0.000015,5.681101e-06,3.468758e-07,3.406447e-07,3.353094e-07,-0.000027,-3.464669e-05,-3.462892e-05,-3.461000e-05,1.301345e-05,...,3.882309e-06,3.938809e-06,3.995313e-06,5.588167e-05,5.595497e-05,-1.911421e-04,-2.294227e-04,-2.042725e-04,-2.042793e-04,-2.044297e-04
112.1,-0.000015,5.674249e-06,3.475334e-07,3.413091e-07,3.359798e-07,-0.000027,-3.460605e-05,-3.458834e-05,-3.456943e-05,1.299776e-05,...,3.885730e-06,3.942425e-06,3.999126e-06,5.581055e-05,5.588379e-05,-1.908943e-04,-2.042297e-04,-2.094143e-04,-2.094222e-04,-2.095789e-04
113.1,-0.000015,5.664689e-06,3.452404e-07,3.390277e-07,3.337082e-07,-0.000027,-3.454593e-05,-3.452818e-05,-3.450933e-05,1.297585e-05,...,3.866321e-06,3.922503e-06,3.978688e-06,5.572241e-05,5.579549e-05,-1.905999e-04,-2.039187e-04,-2.090983e-04,-2.318724e-04,-2.320538e-04
